Notebook dealing with the final preprocessing steps for the neural networks.


In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.utils.data
import pandas as pd
from torch.utils import data
from numpy import array
from numpy import argmax
import argparse
from torch.autograd import Variable
from torch import optim
import numpy as np
import os
#from sklearn.model_selection import train_test_split
import logging
import pickle as pkl
import warnings
pd.options.mode.chained_assignment = None
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import glob
import random
random.seed(1991)
#torch.set_default_tensor_type('torch.cuda.DoubleTensor')

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.manual_seed(1991)
if not torch.cuda.is_available():
    
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda")

In [5]:
heads=pd.read_csv("/data/dharp/compounds/datasets/google_big/heads_reduced.csv",sep="\t",header=None)
#heads=heads.drop(1,axis=1).groupby(0).mean()
#heads.columns=list(range(0,300))
#heads.index.name="head"
#heads.reset_index(inplace=True)
heads=heads.iloc[:,0:52]
heads=pd.pivot_table(heads, index=0, columns=1)
heads.fillna(0,inplace=True)
heads_final_1=heads.columns.get_level_values(0)
heads_final_2=heads.columns.get_level_values(1)
cur_year=0
new_columns=[]
for year in heads_final_2:
    new_columns.append(str(year)+"_"+str(heads_final_1[cur_year]))
    cur_year+=1
heads.columns=new_columns
heads.index.name="head"
heads.reset_index(inplace=True)
heads

,head,1950_2,1960_2,1970_2,1980_2,1990_2,2000_2,1950_3,1960_3,1970_3,...,1970_50,1980_50,1990_50,2000_50,1950_51,1960_51,1970_51,1980_51,1990_51,2000_51
0,#,0.004395,0.005209,0.010188,0.027986,0.056439,0.005607,0.023386,0.027442,0.050353,...,-0.009078,-0.055409,-0.115572,0.014345,0.008747,0.011231,0.008701,0.018684,0.036857,0.005098
1,%,0.019223,0.026601,0.026044,0.031581,0.031332,0.034898,0.139727,0.186300,0.170950,...,-0.097086,-0.082608,-0.100635,-0.077894,0.008245,0.007363,0.009187,0.007340,0.001244,-0.006027
2,',0.012407,0.013071,0.010933,0.026258,0.026893,0.034290,0.074766,0.079966,0.066670,...,-0.018765,-0.040313,-0.131396,-0.096296,0.025964,0.022138,0.018055,0.030136,0.064600,0.053312
3,'s,0.003450,0.005087,0.004633,0.005511,0.007278,0.008235,0.022611,0.033937,0.031035,...,-0.036947,-0.038013,-0.027747,-0.033043,0.027161,0.026264,0.027844,0.027131,0.023814,0.026244
4,*,0.026181,0.025313,0.017901,0.019033,0.019412,0.012768,0.173769,0.162138,0.115458,...,-0.007481,-0.020382,-0.030718,-0.028543,0.023452,0.024262,0.014594,0.016966,0.017860,0.014314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39181,♦,0.029070,0.029357,0.029191,0.029627,0.004249,0.001731,0.066297,0.069229,0.067419,...,0.108787,0.100478,0.005818,-0.001442,-0.012446,-0.012108,-0.012699,-0.012265,0.007739,0.008430
39182,➪,0.000000,0.000000,0.000000,0.009572,0.000000,0.010199,0.000000,0.000000,0.000000,...,0.000000,-0.014762,0.000000,-0.028962,0.000000,0.000000,0.000000,0.011304,0.000000,0.005288
39183,,0.000000,0.000000,0.000000,0.000000,0.000000,0.007409,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.017088,0.000000,0.000000,0.000000,0.000000,0.000000,0.010601
39184,,0.000000,0.000000,0.000000,0.011421,0.011421,0.011421,0.000000,0.000000,0.000000,...,0.000000,-0.013562,-0.013562,-0.013562,0.000000,0.000000,0.000000,0.010543,0.010543,0.010543


In [6]:
modifiers=pd.read_csv("/data/dharp/compounds/datasets/google_big/modifiers_reduced.csv",sep="\t",header=None)
modifiers=modifiers.iloc[:,0:52]
modifiers=pd.pivot_table(modifiers, index=0, columns=1)
modifiers.fillna(0,inplace=True)
modifiers_final_1=modifiers.columns.get_level_values(0)
modifiers_final_2=modifiers.columns.get_level_values(1)
cur_year=0
new_columns=[]
for year in modifiers_final_2:
    new_columns.append(str(year)+"_"+str(modifiers_final_1[cur_year]))
    cur_year+=1
modifiers.columns=new_columns
modifiers.index.name="modifier"
modifiers.reset_index(inplace=True)
modifiers

,modifier,1950_2,1960_2,1970_2,1980_2,1990_2,2000_2,1950_3,1960_3,1970_3,...,1970_50,1980_50,1990_50,2000_50,1950_51,1960_51,1970_51,1980_51,1990_51,2000_51
0,#,0.007835,0.007134,0.006889,0.026035,0.023219,0.009480,0.060462,0.056907,0.056524,...,-0.088560,-0.090730,-0.049824,-0.045186,-0.035971,-0.036821,-0.036994,0.001955,0.009308,-0.000036
1,%,0.067067,0.054705,0.043711,0.040133,0.042022,0.040310,0.214428,0.204628,0.207093,...,-0.099150,-0.087209,-0.072813,-0.061840,-0.039409,-0.042373,-0.041281,-0.038172,-0.034718,-0.034274
2,',0.057065,0.044029,0.020618,0.015135,0.011254,0.018267,0.266026,0.219794,0.117727,...,0.037909,0.029501,0.007262,0.000411,0.369101,0.484699,0.710151,0.393650,0.267442,0.363134
3,'s,0.000000,0.008705,0.000000,0.007716,0.007958,0.009910,0.000000,0.050162,0.000000,...,0.000000,-0.030793,-0.031805,-0.119389,0.000000,0.025053,0.000000,0.006867,0.006625,0.025883
4,*,0.034536,0.022330,0.009456,0.006639,0.012478,0.015028,0.084238,0.092160,0.061693,...,-0.140811,-0.123645,-0.112324,-0.072399,0.046942,0.058971,0.053157,0.042992,0.048521,0.037779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36366,◦,0.000000,0.000000,0.000595,0.000000,0.001067,0.001088,0.000000,0.000000,0.004050,...,0.021025,0.000000,0.063767,0.048157,0.000000,0.000000,0.002344,0.000000,-0.039254,-0.026293
36367,♦,0.014853,0.013009,0.011507,0.002575,0.017410,0.017497,0.051939,0.049263,0.046989,...,-0.089204,-0.060438,-0.097279,-0.062748,0.005296,0.011137,0.017370,0.012557,0.012707,0.014271
36368,➪,0.000000,0.000000,0.000000,0.002429,0.000000,0.003379,0.000000,0.000000,0.000000,...,0.000000,-0.015433,0.000000,-0.011813,0.000000,0.000000,0.000000,0.000089,0.000000,-0.000366
36369,,0.000000,0.000000,0.000000,0.003485,0.004317,0.004429,0.000000,0.000000,0.000000,...,0.000000,-0.018369,-0.021893,-0.023529,0.000000,0.000000,0.000000,0.013451,0.014971,0.015797


In [7]:
compounds=pd.read_csv("/data/dharp/compounds/datasets/google_big/compounds_reduced.csv",sep="\t",header=None)
compounds

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
0,'s,quilt,1990,0.007832,0.059829,-0.031936,-0.066304,0.007579,-0.042816,-0.015612,...,-0.000179,0.029610,-0.005688,-0.025025,-0.016804,-0.016301,0.007180,0.046095,0.050832,-0.006269
1,+1,v,1990,0.010044,0.095959,-0.005823,-0.056895,0.016886,-0.085580,-0.028374,...,-0.032750,0.036233,0.023574,0.038533,0.016283,0.072119,-0.070514,-0.032994,0.006339,-0.031224
2,+5,v,1980,0.021831,0.225970,0.050935,0.010884,-0.021162,0.068693,0.115418,...,0.024447,0.031855,0.047127,0.010924,-0.036636,0.041017,-0.031380,-0.041131,-0.050458,-0.026649
3,+5,v,1990,0.020505,0.206646,0.038952,-0.000764,-0.019165,0.060420,0.106927,...,0.019837,0.026725,0.041276,0.020125,-0.019265,0.053320,-0.016862,-0.055837,-0.079821,-0.029404
4,+5,v,2000,0.024218,0.258608,0.067927,0.025135,-0.024364,0.081789,0.129163,...,0.034007,0.035541,0.055252,0.006387,-0.054912,0.033273,-0.044528,-0.028714,-0.018798,-0.027256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861937,zymogen,granule,2000,0.013367,0.092165,-0.042929,-0.091425,0.005232,-0.029679,0.000581,...,0.027478,0.020900,-0.088507,0.002488,0.072503,0.070204,-0.011515,0.063538,-0.066650,0.008026
861938,¼,inch,2000,0.005826,0.045735,0.001790,-0.015764,0.005044,-0.025822,-0.000190,...,-0.018226,0.071838,-0.003438,0.053666,0.038193,0.071045,-0.063428,-0.034716,0.015961,-0.026191
861939,¼,teaspoon,2000,0.004635,0.042301,-0.008956,-0.032373,0.001632,-0.017565,0.001033,...,-0.106843,0.005386,-0.032731,0.127252,-0.046452,0.026440,0.059794,0.021990,0.001104,-0.143370
861940,½,cup,2000,0.005795,0.037588,-0.027624,-0.021103,0.004270,-0.017355,-0.005591,...,-0.030345,0.057428,0.024897,0.015057,0.138693,-0.005621,0.056439,-0.003240,0.033509,0.018779


In [8]:
novel_compounds=pd.DataFrame.from_records(compounds[[0,1]].value_counts().index.to_list())
novel_compounds.columns=['modifier','head']
novel_compounds

,modifier,head
0,succeeding,year
1,la,rabida
2,christmas,party
3,la,tene
4,la,scala
...,...,...
318168,learning,alliances
318169,learning,area
318170,learning,association
318171,learning,automaton


In [9]:
compounds=novel_compounds.modifier+" "+novel_compounds['head']
compounds_lst=compounds.to_list()

In [12]:
len(compounds_lst)

318173

In [18]:
positive_heads=pd.merge(novel_compounds.drop('modifier',axis=1),heads,on=["head"],how='left')

positive_modifiers=pd.merge(novel_compounds.drop('head',axis=1),modifiers,on=["modifier"],how='left')

In [19]:
to_remove_mods=positive_modifiers[positive_modifiers.isna().any(axis=1)].index.to_list()
to_remove_heads=positive_heads[positive_heads.isna().any(axis=1)].index.to_list()
len(to_remove_mods+to_remove_heads)

270

In [20]:
positive_modifiers.drop(to_remove_mods+to_remove_heads,inplace=True)
positive_heads.drop(to_remove_mods+to_remove_heads,inplace=True)

In [21]:
novel_compounds.drop(to_remove_mods+to_remove_heads,inplace=True)

In [22]:
assert positive_modifiers.shape[0]==positive_heads.shape[0]==novel_compounds.shape[0]

In [23]:
positive_heads_tensor = torch.tensor(positive_heads.drop('head',axis=1).values)
positive_heads_tensor.shape

torch.Size([317903, 300])

In [24]:
positive_modifiers_tensor = torch.tensor(positive_modifiers.drop('modifier',axis=1).values)
positive_modifiers_tensor.shape

torch.Size([317903, 300])

In [25]:
positive_Y=torch.ones(positive_modifiers_tensor.shape[0])
positive_Y.shape

torch.Size([317903])

In [26]:
positive_class=torch.cat((positive_modifiers_tensor, positive_heads_tensor), 1)
positive_class.shape

torch.Size([317903, 600])

In [21]:
negative_heads_tensor = torch.tensor(positive_modifiers.drop('modifier',axis=1).values)
negative_heads_tensor.shape

torch.Size([317903, 300])

In [22]:
negative_modifiers_tensor = torch.tensor(positive_heads.drop('head',axis=1).values)
negative_modifiers_tensor.shape

torch.Size([317903, 300])

In [23]:
negative_Y=torch.zeros(negative_modifiers_tensor.shape[0])
negative_class=torch.cat((negative_modifiers_tensor, negative_heads_tensor), 1)

In [24]:
tensor_list=[]

X=torch.cat((positive_class, negative_class), 0)
Y=torch.cat((positive_Y,negative_Y),0)
tensor_list.append([X,Y])

In [25]:
tensor_list[0][0].shape

torch.Size([635806, 600])

In [26]:
tensor_list[0][1].shape

torch.Size([635806])

In [27]:
input_size = 600
hidden_size = 300
num_classes = 2
num_epochs = 50
batch_size = 100
learning_rate = 0.001

In [28]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes,bias=False)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [29]:
    for dataset in tensor_list:
        X=dataset[0]
        Y=dataset[1]
        n = len(X)  # how many total elements you have
        n_test = int( n * .2 )  # number of test/val elements
        n_train = n - n_test
        idx = list(range(n))  # indices to all elements
        random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
        train_idx = idx[:n_train]
        test_idx = idx[n_train:]
        trX=X[train_idx].float().to(device)
        teX=X[test_idx].float().to(device)

        trY=Y[train_idx].long().to(device)
        teY=Y[test_idx].long().to(device)
        model = NeuralNet(input_size, hidden_size, num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        n_examples=trX.shape[0]
        for i in range(num_epochs):

            cost = 0.

            num_batches = n_examples // batch_size
            for k in range(num_batches):
                start, end = k * batch_size, (k + 1) * batch_size
                outputs = model(trX[start:end])
                loss = criterion(outputs, trY[start:end])

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if (k+1) % 10 == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, num_epochs, loss.item()))

        with torch.no_grad():
            correct=0
            total=0
            outputs = model(teX)
            score, predicted = torch.max(outputs.data, 1)
            total += teY.size(0)
            correct += (predicted == teY).sum().item()
        curr_acc=100 * correct / total
        print(curr_acc)

97.10131250933856


In [30]:
reversed_df=novel_compounds.copy()
novel_compounds['plausible']=1
reversed_df.columns=['head','modifier']
reversed_df['plausible']=0
total_df=pd.concat([novel_compounds,reversed_df],ignore_index=True,sort=False)
total_df

,modifier,head,plausible
0,succeeding,year,1
1,la,rabida,1
2,christmas,party,1
3,la,tene,1
4,la,scala,1
...,...,...,...
635801,alliances,learning,0
635802,area,learning,0
635803,association,learning,0
635804,automaton,learning,0


In [ ]:
for 

In [32]:
testset=total_df.iloc[test_idx]
testset['score']=score.cpu().data.numpy()
testset['predicted']=predicted.cpu().data.numpy()
testset['teY']=teY.cpu().data.numpy()
testset.to_csv('/data/dharp/compounds/datasets/google_big_testset.csv',index=False,header=True,sep="\t")

In [158]:
testset['compound']=testset['modifier']+" "+testset['head']
testset.loc[(testset.compound.isin(compounds_lst))&(testset.predicted==1) &(testset.plausible==0)]

,modifier,head,plausible,score,predicted,teY,compound
523213,wind,rent,0,1.866492,1,0,wind rent
540169,vasodilation,vasodilation,0,1.803424,1,0,vasodilation vasodilation
348972,dune,sand,0,0.139234,1,0,dune sand
610550,pfeiffer,pfeiffer,0,0.608447,1,0,pfeiffer pfeiffer
414456,cloth,sack,0,5.690154,1,0,cloth sack
527029,britain,working,0,0.895440,1,0,britain working
581503,altimeters,altimeters,0,0.669924,1,0,altimeters altimeters
327928,walla,walla,0,2.456021,1,0,walla walla
516933,vote,choice,0,0.648080,1,0,vote choice
563090,filename,filename,0,3.548040,1,0,filename filename


In [144]:
testset[['plausible','predicted']].value_counts(normalize=True)*100

plausible  predicted
1          1            48.681593
0          0            48.478700
           1             1.643586
1          0             1.196121
dtype: float64

In [33]:
model

NeuralNet(
  (fc1): Linear(in_features=600, out_features=300, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=300, out_features=2, bias=False)
)

In [145]:
testset.loc[(testset.plausible==0) & (testset.predicted==1)].to_csv('/data/dharp/compounds/datasets/google_big_fp.csv',index=False,header=True,sep="\t")

In [140]:
np.mean(testset.plausible==testset.predicted)

0.9716029285708668

In [ ]:
torch.save(model.state_dict(), "/data/dharp/compounds/datasets/google_big/googleB.pt")

In [34]:
with open('/data/dharp/compounds/datasets/compound_testset.txt','r') as f:
    novel_compounds_list=f.read().split("\n")
novel_compounds_list=novel_compounds_list[:-1]
len(novel_compounds_list)

12572

In [35]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds[['modifier','head']]=novel_compounds[0].str.split(" ",expand=True)
novel_compounds.drop(0,axis=1,inplace=True)
novel_compounds

,modifier,head
0,#,husky
1,#,nat
2,#,new
3,#,region
4,#,text
...,...,...
12567,•,switzerland
12568,•,transportation
12569,•,type
12570,•,water


In [36]:
positive_heads=pd.merge(novel_compounds.drop('modifier',axis=1),heads,on=["head"],how='left')

positive_modifiers=pd.merge(novel_compounds.drop('head',axis=1),modifiers,on=["modifier"])

In [37]:
to_remove=positive_heads[positive_heads.isna().any(axis=1)].index.values[0]
to_remove

10099

In [38]:
positive_modifiers.drop(to_remove,inplace=True)
positive_heads.drop(to_remove,inplace=True)
novel_compounds.drop(to_remove,inplace=True)

In [42]:
total_df['common']=total_df.modifier+" "+total_df['head']
novel_compounds['common']=novel_compounds.modifier+" "+novel_compounds['head']

In [48]:
not_novel_comps=total_df.loc[total_df.common.isin(novel_compounds.common.to_list())]['common'].to_list()
len(not_novel_comps)

3152

In [51]:
novel_compounds=novel_compounds.loc[~novel_compounds.common.isin(not_novel_comps)]

In [56]:
positive_heads=pd.merge(novel_compounds.drop(['modifier','common'],axis=1),heads,on=["head"],how='left')

positive_modifiers=pd.merge(novel_compounds.drop(['head','common'],axis=1),modifiers,on=["modifier"])
positive_modifiers

,modifier,1950_2,1960_2,1970_2,1980_2,1990_2,2000_2,1950_3,1960_3,1970_3,...,1970_50,1980_50,1990_50,2000_50,1950_51,1960_51,1970_51,1980_51,1990_51,2000_51
0,#,0.007835,0.007134,0.006889,0.026035,0.023219,0.009480,0.060462,0.056907,0.056524,...,-0.088560,-0.090730,-0.049824,-0.045186,-0.035971,-0.036821,-0.036994,0.001955,0.009308,-0.000036
1,#,0.007835,0.007134,0.006889,0.026035,0.023219,0.009480,0.060462,0.056907,0.056524,...,-0.088560,-0.090730,-0.049824,-0.045186,-0.035971,-0.036821,-0.036994,0.001955,0.009308,-0.000036
2,#,0.007835,0.007134,0.006889,0.026035,0.023219,0.009480,0.060462,0.056907,0.056524,...,-0.088560,-0.090730,-0.049824,-0.045186,-0.035971,-0.036821,-0.036994,0.001955,0.009308,-0.000036
3,#,0.007835,0.007134,0.006889,0.026035,0.023219,0.009480,0.060462,0.056907,0.056524,...,-0.088560,-0.090730,-0.049824,-0.045186,-0.035971,-0.036821,-0.036994,0.001955,0.009308,-0.000036
4,#,0.007835,0.007134,0.006889,0.026035,0.023219,0.009480,0.060462,0.056907,0.056524,...,-0.088560,-0.090730,-0.049824,-0.045186,-0.035971,-0.036821,-0.036994,0.001955,0.009308,-0.000036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9508,•,0.034732,0.030566,0.024899,0.035439,0.042818,0.063467,0.175315,0.190146,0.158476,...,-0.080298,-0.106420,-0.135805,-0.232512,0.019451,0.028502,0.021737,0.011809,0.015086,0.040806
9509,•,0.034732,0.030566,0.024899,0.035439,0.042818,0.063467,0.175315,0.190146,0.158476,...,-0.080298,-0.106420,-0.135805,-0.232512,0.019451,0.028502,0.021737,0.011809,0.015086,0.040806
9510,•,0.034732,0.030566,0.024899,0.035439,0.042818,0.063467,0.175315,0.190146,0.158476,...,-0.080298,-0.106420,-0.135805,-0.232512,0.019451,0.028502,0.021737,0.011809,0.015086,0.040806
9511,•,0.034732,0.030566,0.024899,0.035439,0.042818,0.063467,0.175315,0.190146,0.158476,...,-0.080298,-0.106420,-0.135805,-0.232512,0.019451,0.028502,0.021737,0.011809,0.015086,0.040806


In [78]:
positive_heads_tensor = torch.tensor(positive_heads.drop('head',axis=1).values)
positive_heads_tensor.shape

torch.Size([9513, 300])

In [79]:
positive_modifiers_tensor = torch.tensor(positive_modifiers.drop('modifier',axis=1).values)
positive_modifiers_tensor.shape

torch.Size([9513, 300])

In [81]:
positive_class=torch.cat((positive_modifiers_tensor, positive_heads_tensor), 1)
positive_class.shape

torch.Size([9513, 600])

In [80]:
positive_Y=torch.ones(positive_modifiers_tensor.shape[0])
positive_Y.shape

torch.Size([9513])

In [82]:
teX=positive_class.float().to(device)

In [83]:
            outputs = model(teX)
            score, predicted = torch.max(outputs.data, 1)

In [84]:
teX.shape

torch.Size([9513, 600])

In [85]:
score.shape

torch.Size([9513])

In [86]:
predicted.shape

torch.Size([9513])

In [89]:
novel_compounds

,modifier,head,common,score,predicted
0,#,husky,# husky,17.500648,1
1,#,nat,# nat,11.575470,1
2,#,new,# new,9.348957,1
3,#,region,# region,42.051151,1
4,#,text,# text,22.072235,1
...,...,...,...,...,...
12567,•,switzerland,• switzerland,2.924459,1
12568,•,transportation,• transportation,19.159500,1
12569,•,type,• type,19.596848,1
12570,•,water,• water,11.413174,1


In [90]:
novel_compounds['score']=score.cpu().data.numpy()
novel_compounds['predicted']=predicted.cpu().data.numpy()
novel_compounds['teY']=1
novel_compounds

,modifier,head,common,score,predicted,teY
0,#,husky,# husky,17.500648,1,1
1,#,nat,# nat,11.575470,1,1
2,#,new,# new,9.348957,1,1
3,#,region,# region,42.051151,1,1
4,#,text,# text,22.072235,1,1
...,...,...,...,...,...,...
12567,•,switzerland,• switzerland,2.924459,1,1
12568,•,transportation,• transportation,19.159500,1,1
12569,•,type,• type,19.596848,1,1
12570,•,water,• water,11.413174,1,1


In [93]:
novel_compounds.to_csv('/data/dharp/compounds/datasets/google_actual_pred.csv',index=False,header=True,sep="\t")

In [92]:
novel_compounds.predicted.value_counts(normalize=True)

1    0.965521
0    0.034479
Name: predicted, dtype: float64

In [ ]:
novel_compounds.loc[novel_compounds.predicted==1

In [ ]:
new_testset=total_df.iloc[test_idx]
testset['score']=score.cpu().data.numpy()
testset['predicted']=predicted.cpu().data.numpy()
testset['teY']=teY.cpu().data.numpy()
testset.to_csv('/data/dharp/compounds/datasets/google_big_testset.csv',index=False,header=True,sep="\t")